# Lesson 14 Studio Solution:  SQL Part 5

Studio notebook: https://github.com/speudusa/SQL-Part-5-Studio/blob/master/Studio.ipynb

## Part B: WARM-UP QUERIES

In [1]:
USE JunkDB_STL_May2023
GO

-- WARM UP --

-- Query 1 -- Return the mystery book titles and their ISBNs.

/* Our book titles and ISBNs are all in the book table, but the genre is
an ID number that references the genre table. So, how can we get those Mystery IDs,
and use those IDs to filter the book table? There are several ways to do this!*/

-- SUBQUERY METHOD - First get the Mystery IDs, then use those IDs to filter the Book table

-- Total execution time: 00:00:00.172
    SELECT b.title, b.isbn
    FROM helena.book AS b 
    WHERE b.genre_id IN     
        (SELECT genre_id    
        FROM helena.genre 
        WHERE genres LIKE 'Mystery');


-- INNER JOIN METHOD - Filter the Genre table down to only Mystery IDs, then join the results to
    -- the book table to get only the books where Genre ID is in the mystery category

    -- Total execution time: 00:00:00.146
    SELECT b.title, b.isbn
    FROM helena.book AS b
    INNER JOIN helena.genre AS g
    ON b.genre_id = g.genre_id
    WHERE g.genres LIKE 'Mystery';

/* Inner join runs in about 85% of the time it takes to run the subquery method, so
if we were writing this query to run on a very large dataset or it were a query we knew
would run very frequently, it'd be wise to choose the inner join method instead of the
subquery method. If it's a query you're running just one time for an analysis and your
dataset isn't massive, this sort of efficiency analysis is less important */

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.066

In [12]:
-- Query 2 -- Return all of the titles and the author's first and last names 
            --for books written by authors who are currently living.

/* If we examine our author data, we can see that living authors have
NULL values recorded in the death_year column: */

SELECT first_name, last_name, birth_year, death_year FROM helena.author;

/* So, if we want to return the first and last names of the authors who are
currently alive, we can simply filter out any records with data in the
death_year column: */

SELECT first_name, last_name 
FROM helena.author
WHERE death_year IS NULL;

/* The request is also asking for book titles, which we can only get from the book table,
Unlike the Mystery genre exercise, where we only needed to return data from one table,
in this instance we want to return data from both the author table AND the book table.
Since we need data from both, we're going to use a join instead of a subquery. */

SELECT b.title, a.first_name, a.last_name -- get the title from the book table and first/last name from the author table
FROM helena.book AS b -- alias your tables to make your life easier
INNER JOIN helena.author AS a -- inner join so we only keep things that match
ON b.author_id = a.author_id -- tell SQL which columns to match on
WHERE a.death_year IS NULL; -- and apply our filter

## **Part C: Loan Out a Book**

## 

The library would like you to create a query that will update the database when a book is loaned out.  

This query needs to perform the following functions:

1. Change **available** to **0** (false) for the appropriate book.
    
2. Add a new row to the **loan** table with today's date as the **date\_out** and the ids in the row matching the appropriate **patron\_id** and **book\_id**.
    
3. Update the appropriate **patron** with the **loan\_id** for the new row created in the **loan** table.
    

You can use any patron and book that strikes your fancy to create and test out this query.'

> _Note:  Every time you test this, you will add an additional row to the output, creating a new loan id each time.  If you do not update the patron id, then a patron will have multiple loan ids.  This is not an issue for Part C;_ _however, this may cause issues when you work on Part F._

[https://stackoverflow.com/questions/224732/sql-update-from-one-table-to-another-based-on-a-id-match](https://stackoverflow.com/questions/224732/sql-update-from-one-table-to-another-based-on-a-id-match)

[https://docs.microsoft.com/en-us/sql/t-sql/queries/update-transact-sql?view=sql-server-ver15](https://docs.microsoft.com/en-us/sql/t-sql/queries/update-transact-sql?view=sql-server-ver15)

In [35]:
/* For the sake of example, we're going to use book_id 10, "Sense and Sensibility" */

SELECT * from helena.book where book_id = 10 -- Note that the available column is currently 1, 
                                                -- meaning it's available to checkout
/* And patron_id 5, "Wren Gergler" */

SELECT * from helena.patron where patron_id = 5 

/* Wren is going to check out Sense and Sensibility, so we need to change that book's availability
to 0. */

UPDATE helena.book
SET available = 0
WHERE book_id = 10; 

SELECT * from helena.book where book_id = 10

/* Now, we're going to document that Wren checked the book out on the loan table. To do this, we will
create a new row on the loan table with Wren's patron ID, Sense and Sensibility's book ID, and we'll
document that the book was checked out today in the date_out column. */

INSERT INTO helena.loan (patron_id, date_out, book_id)
VALUES (5,  -- Wren's patron ID
        GETDATE(), -- today's date
        10); -- Sense and Sensibility's book ID 

SELECT * FROM helena.loan;


/* Now we need to get the ID we just generated by loaning out Sense and Sensibility to Wren and
add it to the patron table so we can cross-reference. */

UPDATE helena.patron
SET helena.patron.loan_id = (
    SELECT helena.loan.loan_id
    FROM helena.loan
    WHERE helena.loan.patron_id = helena.patron.patron_id
);

SELECT * FROM helena.patron WHERE patron_id = 5;

Check out five more books to five more patrons

In [36]:
-- Patron 1, Book 1

UPDATE helena.book
SET available = 0
WHERE book_id = 1; 

INSERT INTO helena.loan (patron_id, date_out, book_id)
VALUES (1,  -- patron ID
        GETDATE(), -- today's date
        1); -- book ID 

UPDATE helena.patron
SET helena.patron.loan_id = (
    SELECT helena.loan.loan_id
    FROM helena.loan
    WHERE helena.loan.patron_id = helena.patron.patron_id
);

-- Patron 2, Book 2

UPDATE helena.book
SET available = 0
WHERE book_id = 2; 

INSERT INTO helena.loan (patron_id, date_out, book_id)
VALUES (2,  -- patron ID
        GETDATE(), -- today's date
        2); -- book ID 

UPDATE helena.patron
SET helena.patron.loan_id = (
    SELECT helena.loan.loan_id
    FROM helena.loan
    WHERE helena.loan.patron_id = helena.patron.patron_id
);

-- Patron 3, Book 3

UPDATE helena.book
SET available = 0
WHERE book_id = 3; 

INSERT INTO helena.loan (patron_id, date_out, book_id)
VALUES (3,  -- patron ID
        GETDATE(), -- today's date
        3); -- book ID 

UPDATE helena.patron
SET helena.patron.loan_id = (
    SELECT helena.loan.loan_id
    FROM helena.loan
    WHERE helena.loan.patron_id = helena.patron.patron_id
);

-- Patron 4, Book 4

UPDATE helena.book
SET available = 0
WHERE book_id = 4; 

INSERT INTO helena.loan (patron_id, date_out, book_id)
VALUES (4,  -- patron ID
        GETDATE(), -- today's date
        4); -- book ID 

UPDATE helena.patron
SET helena.patron.loan_id = (
    SELECT helena.loan.loan_id
    FROM helena.loan
    WHERE helena.loan.patron_id = helena.patron.patron_id
);

-- Patron 5, Book 5

UPDATE helena.book
SET available = 0
WHERE book_id = 6; 

INSERT INTO helena.loan (patron_id, date_out, book_id)
VALUES (6,  -- patron ID
        GETDATE(), -- today's date
        6); -- book ID 

UPDATE helena.patron
SET helena.patron.loan_id = (
    SELECT helena.loan.loan_id
    FROM helena.loan
    WHERE helena.loan.patron_id = helena.patron.patron_id
);

In [44]:
/* Let's look at the checked out books real quick */

SELECT b.title, 
    a.first_name + ' ' + a.last_name AS 'author', 
    p.first_name + ' ' + p.last_name AS 'patron', 
    l.date_out, l.date_in 
FROM helena.loan AS l
JOIN helena.book AS b on b.book_id = l.book_id
JOIN helena.author AS a on a.author_id = b.author_id
JOIN helena.patron AS p on p.patron_id = l.patron_id;

## **Part D: Check a Book Back In**

Work with the same patron and book as you create your query to check a book back into the library.  

The check a book back in, your query need to do the following:

1. Change **available** to **1** (true) for the appropriate book.
    
2. Update the appropriate row in the **<mark><mark><mark>loan</mark></mark></mark>** table with today's date as the **date\_in**.
    
3. Update the appropriate **patron** changing **<mark><mark><mark>loan</mark></mark></mark>\_id** back to **NULL**.

In [27]:
/* Now we're going to check Wren's book back into the library. */

-- Mark the book "available" again in the book table- remember, our book ID is 10
SELECT * FROM helena.book WHERE book_id = 10;

UPDATE helena.book
SET available = 1
WHERE book_id = 10;

SELECT * FROM helena.book WHERE book_id = 10;

-- Update date_in column on the loan table to reflect that the book was returned today
SELECT * FROM helena.loan WHERE book_id = 10;

UPDATE helena.loan
SET date_in = GETDATE()
WHERE date_in IS NULL;

SELECT * FROM helena.loan WHERE book_id = 10;

/* Lastly, we just have to switch Wren's loan_id to NULL since she no longer has a book out. */

SELECT * FROM helena.patron WHERE patron_id = 5;

UPDATE helena.patron
SET loan_id = NULL
WHERE patron_id = 5;

SELECT * FROM helena.patron WHERE patron_id = 5;

## **Part E: Create a temp table**

## 

Write a query that returns a temporary table for a patron that shows them all of their loans.

_Question to think about:_  Why might a temp table be a good option for this type of query?

In [65]:
-- TEMP TABLE

/* Start by thinking of what you would want to see in this table if you were a patron 
trying to check on your loans. You'd probably want to see the book's title and when
you checked it out, right? So let's logic through how we'll build this table.

We get our information about books on loan from the loan table.
We get book titles from the book table.
We get information about patrons from the patron table. 
We know a book has not been checked in if the date_in column on the loan table is NULL.

We know that the loan table has a book ID and a patron ID, so we can start with the
loan table and then join our remaining data onto it. 

Lets get all rows from the loan table where:
- Wren is the patron (patron_id = 5)
- The book is currently checked out (date_in = NULL) 

We know we'll be joining this with other tables, so let's go ahead
and start writing aliases now. */

SELECT * 
FROM helena.loan AS l
WHERE l.patron_id = 5 AND l.date_in IS NULL;

/* Now let's get Wren's actual name into the table instead of just their ID. To make
it easier for our end user to read, let's combine the first_name and last_name
columns from the patron table into a single column called "patron." */

SELECT  p.first_name + ' ' + p.last_name AS 'patron', 
        l.*
FROM helena.patron AS p
INNER JOIN helena.loan AS l ON p.patron_id = l.patron_id
WHERE p.patron_id = 5 AND l.date_in IS NULL;

/* Now we'll add the book title from the book table. */

SELECT  p.first_name + ' ' + p.last_name AS 'patron', 
        b.title, 
        l.*
FROM helena.patron AS p
INNER JOIN helena.loan AS l ON p.patron_id = l.patron_id
INNER JOIN helena.book AS b on l.book_id = b.book_id
WHERE p.patron_id = 5 AND l.date_in IS NULL;

/* Let's get rid of the unnecessary columns from the loan table. All Wren needs
to see from that table is the checkout date for each book. We can still perform
joins on the ID columns even if we can't see them on the final table!*/

SELECT  p.first_name + ' ' + p.last_name AS 'patron', 
        b.title, 
        l.date_out
FROM helena.patron AS p
INNER JOIN helena.loan AS l ON p.patron_id = l.patron_id
INNER JOIN helena.book AS b on l.book_id = b.book_id
WHERE p.patron_id = 5 AND l.date_in IS NULL;

/* We may need to query this table later. However, we don't want to create
and save this new table EVERY time a different patron looks at their data, right?
So, we can create Temp Table that can be queried inside of this SQL document without
rewriting the ENTIRE query every time we want to use that table. */

DROP TABLE IF EXISTS #TempTest1;

SELECT  p.first_name + ' ' + p.last_name AS 'patron', 
        b.title, 
         b.author,
        l.date_out
INTO #TempTest1
FROM helena.patron AS p
INNER JOIN helena.loan AS l ON p.patron_id = l.patron_id
INNER JOIN helena.book AS b on l.book_id = b.book_id
WHERE p.patron_id = 5 AND l.date_in IS NULL;

/* Look over at the Tables section in your database navigator- "#TempTest1" doesn't exist there.
It only exists in this session. But, you can still query it.*/

SELECT COUNT(*) AS 'Total # of books currently checked out' FROM #TempTest1;

/* Imagine you're building a library app- it might be useful to have a
query like this that any user can access in order to see the books
they currently have checked out- in fact, this is exactly the kind of query
your library likely uses to show you YOUR checked-out books when you check
your account.


## **Part F: Wrap-up Query**

## 

Create a query that returns the names of the patrons with the genre of every book they currently have checked out.

Things to consider:  How are these tables related and connected to each other?

> _Note:  If you find a NULL in your output instead of a loan id,   this is likely because there are multiple loans per patron.  Try_ _clearing out the loan table.  You can do this by either dropping the entire table or clearing specific rows._

In [68]:

-- retrieve first and last name of patron, and genre of loaned out book

SELECT first_name, last_name, genres
FROM (
    SELECT first_name, last_name, book_id FROM helena.patron -- return first and last name, 
                                                                -- and book id from patron_loan table
    INNER JOIN helena.loan AS l ON l.loan_id = patron.loan_id -- Merge entries from loan and 
                                                                -- patron tables with same loan_ids
    ) AS patron_loan  -- alias of our joined tables
INNER JOIN (
    SELECT book_id, genre_id FROM helena.book
    WHERE available = 0 -- Return entries for books that are NOT available.
    ) AS checked_out_genres ON patron_loan.book_id = checked_out_genres.book_id -- Merge entries from 
                                                                                -- patron_loan and 
                                                                                -- checked_out_genres 
                                                                                -- that have matching 
                                                                                -- book_id values.
INNER JOIN helena.genre ON checked_out_genres.genre_id = genre.genre_id; -- Merge entries from genre 
                                                                            -- and checked_out_genres 
                                                                            -- that have mathcing genre_id 
                                                                            -- values.



/*
Note:  If you find a NULL in your output instead of a loan id,
 this is likely because there are multiple loans per patron.  
 Try clearing out the loan table.  
 You can do this by either dropping the entire table or clearing specific rows.
 */

## <span style="font-size: 14px; font-family: -apple-system, BlinkMacSystemFont, sans-serif;">BONUS CHALLENGE</span>  

<span style="font-size: 14px; font-family: -apple-system, BlinkMacSystemFont, sans-serif;"><br></span>

A reference book cannot leave the library.  How would you modify either the **reference\_book** table or the **book** table to make sure that doesn't happen?  Try to apply your modifications.<span style="font-size: 14px; font-family: -apple-system, BlinkMacSystemFont, sans-serif;"><br></span>

In [90]:
/* Let's think through the request and how it relates to the data.

We can modify either reference_books or book. What data in those tables might be useful? */

SELECT TOP 1 * FROM helena.reference_books;
SELECT TOP 1 * FROM helena.book;

/* It might be valuable to consider how we determine if a book is available to check out.
We know that the column "available" in the book table lets us know if a book is available or not.
If a checked-out book shows up in the system as "not available", maybe we could mark all
reference books as checked out so they show up as unavailable. */

/* reference_books lists the book ids of all reference books in the library, 
so maybe getting those book ids would be a good place to start. */

SELECT book_id FROM helena.reference_books;

/* Since the "available" column is on the book table, lets subquery the book table
using the data from the referenc_books query */

SELECT * FROM helena.book
WHERE book_id in (
    SELECT book_id FROM helena.reference_books
    );

/* Now we have our data, let's update the "available" column to 1 to show that the
reference books are unavailable! */

UPDATE helena.book 
SET available = 1
WHERE book_id in (
    SELECT book_id FROM helena.reference_books
    );

/* Let's check our work! */

SELECT * FROM helena.book
WHERE book_id in (
    SELECT book_id FROM helena.reference_books
    );

/* Here's another method using CASE, which we haven't covered in class! 
Read more about CASE here: https://learnsql.com/blog/case-in-sql/ 

CASE works kind of like an IF/ELSE statement in Python.*/

UPDATE helena.book
SET available = CASE -- we're updating the "available" column in helena.book
    WHEN genre_id IN ( -- WHEN acts like IF; here, we're saying "if genre_id..."
        SELECT genre_id from helena.genre WHERE genres = 'Reference' ) --"is in the list of genre id's generated by this subquery..."
    THEN 1 -- THEN is the value if the statement evaluates to TRUE
    ELSE 0 -- ELSE is the result if the statement evaluates to FALSE
    END -- CASE statements use END commands to indicate they're finished instead of semicolons

/* Let's check our work again! */

SELECT * FROM helena.book
WHERE genre_id in (
    SELECT genre_id from helena.genre WHERE genres = 'Reference'
    );

In [1]:
/* Now let's update our check-out query to prevent users from checking out checked-out books.
This is the same query we used earlier, just updated to throw an error if a user attempts to
check out an already-checked-out book. */

-- Patron 7, Book 32

UPDATE helena.book 
SET available = CASE 
    WHEN book_id = 32 AND available = 1
    THEN 0
    ELSE 'Book cannot be checked out'
    END;


INSERT INTO helena.loan (patron_id, date_out, book_id)
VALUES (1,  -- patron ID
        GETDATE(), -- today's date
        1); -- book ID 

UPDATE helena.patron
SET helena.patron.loan_id = (
    SELECT helena.loan.loan_id
    FROM helena.loan
    WHERE helena.loan.patron_id = helena.patron.patron_id
);

: Msg 208, Level 16, State 1, Line 7
Invalid object name 'helena.book'.

Total execution time: 00:00:00.043